In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import random

voice =pd.read_csv('voice.csv')
voice = pd.DataFrame(voice)
print(voice.head())

   meanfreq        sd    median       Q25       Q75       IQR       skew  \
0  0.059781  0.064241  0.032027  0.015071  0.090193  0.075122  12.863462   
1  0.066009  0.067310  0.040229  0.019414  0.092666  0.073252  22.423285   
2  0.077316  0.083829  0.036718  0.008701  0.131908  0.123207  30.757155   
3  0.151228  0.072111  0.158011  0.096582  0.207955  0.111374   1.232831   
4  0.135120  0.079146  0.124656  0.078720  0.206045  0.127325   1.101174   

          kurt    sp.ent       sfm  ...    centroid   meanfun    minfun  \
0   274.402906  0.893369  0.491918  ...    0.059781  0.084279  0.015702   
1   634.613855  0.892193  0.513724  ...    0.066009  0.107937  0.015826   
2  1024.927705  0.846389  0.478905  ...    0.077316  0.098706  0.015656   
3     4.177296  0.963322  0.727232  ...    0.151228  0.088965  0.017798   
4     4.333713  0.971955  0.783568  ...    0.135120  0.106398  0.016931   

     maxfun   meandom    mindom    maxdom   dfrange   modindx  label  
0  0.275862  0.007812

/home/yangzhibo5947/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
for i in range(6):
    copy = voice
    copy['meanfreq']=copy['meanfreq']+random.gauss(.0001,.001) # add noice to mean freq var
    voice=voice.append(copy,ignore_index=True) # make voice df 2x as big
    print("shape of df after {0}th intertion of this loop is {1}".format(i,voice.shape))

shape of df after 0th intertion of this loop is (6336, 21)
shape of df after 1th intertion of this loop is (12672, 21)
shape of df after 2th intertion of this loop is (25344, 21)
shape of df after 3th intertion of this loop is (50688, 21)
shape of df after 4th intertion of this loop is (101376, 21)
shape of df after 5th intertion of this loop is (202752, 21)


In [3]:
label = voice.pop("label")

# converts from dataframe to np array
voice=voice.values

# convert train labels to one hots
train_labels = pd.get_dummies(label)
# make np array
train_labels = train_labels.values

x_train,x_test,y_train,y_test = train_test_split(voice,train_labels,test_size=0.2)
# # so no we have predictors and y values, separated into test and train

x_train,x_test,y_train,y_test = np.array(x_train,dtype='float32'), np.array(x_test,dtype='float32'), np.array(y_train,dtype='float32'), np.array(y_test,dtype='float32')


In [4]:
print("x_train looks like:\n",x_train[1:2],"\nand y train looks like:\n",y_train[1:2])


print("shape of x_train is: \n",x_train.shape)


x_train looks like:
 [[ 0.18717898  0.04334165  0.19327044  0.16509435  0.21396226  0.04886793
   1.6974895   5.56550884  0.89263111  0.3544755   0.19327044  0.18975395
   0.14567631  0.04692082  0.2742857   1.14283288  0.0234375   6.1640625
   6.140625    0.12009586]] 
and y train looks like:
 [[ 1.  0.]]
shape of x_train is: 
 (162201, 20)


In [5]:
# place holder for inputs. feed in later
x = tf.placeholder("float", [None, 20])

# # take 20 features to 1000 nodes in hidden layer. why? just cuz?
w1 = tf.Variable(tf.random_normal([20, 1000],stddev=.5,name='w1'))
# # add biases for each node
b1 = tf.Variable(tf.zeros([1000]))
# calculate activations 
hidden_output = tf.nn.softmax(tf.matmul(x, w1) + b1)

# bring from 10 nodes to 2 for my output
w2 = tf.Variable(tf.random_normal([1000, 2],stddev=.5,name='w2'))

b2 = tf.Variable(tf.zeros([2]))

In [6]:
# placeholder for correct values 
y_ = tf.placeholder("float", [None,2])
# #implement model. these are predicted ys
y = tf.nn.softmax(tf.matmul(hidden_output, w2) + b2)


In [7]:
loss = tf.reduce_mean(tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(y, y_, name='xentropy')))

In [8]:
opt = tf.train.AdamOptimizer(learning_rate=0.001)
train_step = opt.minimize(loss, var_list=[w1,b1,w2,b2])

In [9]:
tf_correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
tf_accuracy = tf.reduce_mean(tf.cast(tf_correct_prediction, "float"))


In [10]:
def get_mini_batch(x,y):
    rows=np.random.choice(x.shape[0], 100)
    return x[rows], y[rows]

In [11]:
# start session
sess = tf.Session()
summary_writer = tf.train.SummaryWriter('voices')
#summary_writer = tf.train.SummaryWriter('voices', sess.graph)

# # init all vars
init = tf.initialize_all_variables()
sess.run(init)

In [12]:
ntrials = 10000
for i in range(ntrials):
    # get mini batch
    a,b=get_mini_batch(x_train,y_train)
    # run train step, feeding arrays of 100 rows each time
    _, cost =sess.run([train_step,loss], feed_dict={x: a, y_: b})
    if i%100 ==0:
    	print("epoch is {0} and cost is {1}".format(i,cost))

epoch is 0 and cost is 69.904296875
epoch is 100 and cost is 67.90319061279297
epoch is 200 and cost is 66.94023132324219
epoch is 300 and cost is 62.849586486816406
epoch is 400 and cost is 62.152748107910156
epoch is 500 and cost is 61.532554626464844
epoch is 600 and cost is 59.95722961425781
epoch is 700 and cost is 57.5360107421875
epoch is 800 and cost is 56.6322021484375
epoch is 900 and cost is 56.70622253417969
epoch is 1000 and cost is 55.18450927734375
epoch is 1100 and cost is 55.0582389831543
epoch is 1200 and cost is 58.599830627441406
epoch is 1300 and cost is 52.8678092956543
epoch is 1400 and cost is 55.328834533691406
epoch is 1500 and cost is 55.005558013916016
epoch is 1600 and cost is 46.590702056884766
epoch is 1700 and cost is 51.926727294921875
epoch is 1800 and cost is 48.08849334716797
epoch is 1900 and cost is 53.03516387939453
epoch is 2000 and cost is 50.501712799072266
epoch is 2100 and cost is 46.83556365966797
epoch is 2200 and cost is 48.89889907836914


In [13]:
print(y_test.shape)

result = sess.run(tf_accuracy, feed_dict={x: x_test, 
                                          y_: y_test})

print("Train accuracy: {}".format(result))

(40551, 2)
Test accuracy: 0.957263708114624


In [16]:
result = sess.run(tf_accuracy, feed_dict={x:x_train,
                                          y_ : y_train})
print("Test accuracy: {}".format(result))

Test accuracy: 0.9562333226203918
